Generate a random float.

In [1]:
import random
import struct
import numpy as np
import torch
import qtorch
from qtorch.quant import posit_quantize
from qtorch.quant import convert_to_posit

testfloat=random.random()*1e3
print(testfloat)

Using /home/himeshi/.cache/torch_extensions/py310_cu126 as PyTorch extensions root...
Emitting ninja build file /home/himeshi/.cache/torch_extensions/py310_cu126/quant_cpu/build.ninja...
Building extension module quant_cpu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/4] c++ -MMD -MF sim_helper.o.d -DTORCH_EXTENSION_NAME=quant_cpu -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1016\" -isystem /home/himeshi/conga25/conga25env/lib/python3.10/site-packages/torch/include -isystem /home/himeshi/conga25/conga25env/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/himeshi/.pyenv/versions/3.10.4/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=1 -fPIC -std=c++17 -O3 -std=c++17 -fPIC -c /home/himeshi/conga25/QPyTorch/qtorch/quant/quant_cpu/sim_helper.cpp -o sim_helper.o 
[2/4] c++ -MMD -MF bit_helper.o.d -DTORCH_EXTENSION_NAME=quant_cpu -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1016\" -isystem /home/himeshi/conga25/conga25env/lib/python3.10/site-packages/torch/include -isystem /home/himeshi/conga25/conga25env/lib/python3.10/site-packages/torch/include/torch/csrc/a

Loading extension module quant_cpu...
Using /home/himeshi/.cache/torch_extensions/py310_cu126 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/himeshi/.cache/torch_extensions/py310_cu126/quant_cuda/build.ninja...
/home/himeshi/conga25/conga25env/lib/python3.10/site-packages/torch/utils/cpp_extension.py:2356: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module quant_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
715.5914759520085


Loading extension module quant_cuda...


Extract sign, exponent and fraction

In [2]:
def float_to_bits(f):
    # Pack float into 8 bytes, then unpack as 64-bit unsigned int
    packed = struct.pack('>d', f)  # Big endian double
    integer_representation = int.from_bytes(packed, byteorder='big')
    return integer_representation

def extract_components(f):
    bits = float_to_bits(f)

    # Extract sign (1 bit), exponent (11 bits), and fraction (52 bits)
    sign = (bits >> 63) & 0x1
    exponent = (bits >> 52) & 0x7FF
    fraction = bits & ((1 << 52) - 1)

    # Compute values
    sign_val = (-1) ** sign
    exponent_val = exponent - 1023  # Bias for double-precision is 1023
    print(type(1 * (2 ** -(1 + 1))))
    fraction_val = 1 + sum(
        [((fraction & (1 << (52 - i))) >> (51 - i)) * (1 * (2 ** -(i + 1))) for i in range(52)]
    ) if exponent != 0 else sum(
        [((fraction & (1 << (52 - i))) >> (51 - i)) * 1 * (2 ** -(i + 1)) for i in range(52)]
    )

    # Format binary strings
    sign_bits = f"{sign:b}"
    exponent_bits = f"{exponent:011b}"
    fraction_bits = f"{fraction:052b}"

    print(f"Input float: {f}")
    print(f"Sign bit     : {sign_bits} (value: {sign_val})")
    print(f"Exponent bits: {exponent_bits} (value: {exponent} => unbiased: {exponent_val})")
    print(f"Fraction bits: {fraction_bits}")
    print(f"Fraction value: {fraction_val}")
    print(f"Reconstructed value (approx): {sign_val * (2 ** exponent_val) * fraction_val}")

extract_components(testfloat)


<class 'float'>
Input float: 715.5914759520085
Sign bit     : 0 (value: 1)
Exponent bits: 10000001000 (value: 1032 => unbiased: 9)
Fraction bits: 0110010111001011101101010111101111100111000111111010
Fraction value: 1.3976396014687666
Reconstructed value (approx): 715.5914759520085


Convert float value to posit and check sign, exponent, fraction.

In [6]:
float_tensor = torch.tensor(testfloat, dtype=torch.float)
print(isinstance(float_tensor, torch.Tensor))
b = convert_to_posit(float_tensor, nsize=16, es=2)
print(b)

True


RuntimeError: expected scalar type UInt16 but found Float